# Import Modules

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import sqlite3

# Météo

## Extract and transform
From website [https://www.meteociel.fr/climatologie](https://www.meteociel.fr/climatologie)

In [2]:
def extractTemperature(text):
    regex = re.compile(r'([\d\.]+)')
    if text == ' --- ':
        return None
    temperature = regex.findall(text)[0]
    return float(temperature)
    
def extractDate(date, month, year):
    regex = re.compile(r'\d+')
    day = regex.findall(date)[0]
    date = f"{day:0>2}-{month:0>2}-{year}"
    return date

In [3]:
def scrapeMonthYear(month, year, dates, temperature_max, temperature_min):
    url = "https://www.meteociel.fr/climatologie/obs_villes.php"
    
    
    params = {
        "code2": 75107005,
        "mois": month,
        "annee": year,
    }

    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.content, "html.parser")

    
    table = soup.find("table", style="border-collapse: collapse")
    try:
        rows = table.find_all("tr")[1:-1]

        for row in rows:
            date = extractDate(row.find("td", bgcolor="#FFFFCC").text, month, year)
            dates.append(date)
            temperature_max.append(extractTemperature(row.find("td", bgcolor="#FFDDDD").text))
            temperature_min.append(extractTemperature(row.find("td", bgcolor="#DDDDFF").text))
            
    except:
        print(f"No data for {month:0>2}/{year}")

In [4]:
year_min = 2015
year_max = 2024

dates = []
temperature_max = []
temperature_min = []

for year in range(year_min, year_max+1):
    for month in range(1, 13):
        scrapeMonthYear(month, year, dates, temperature_max, temperature_min)

df_weather = pd.DataFrame({
                    "date": dates,
                    "temperature max" : temperature_max,
                    "temperature min": temperature_min
        })

No data for 01/2015
No data for 02/2015
No data for 03/2015
No data for 03/2021


In [5]:
#Save dataframe
df_weather.to_csv(f"../data/temperature_{year_min}_{year_max}.csv", index=False)

# Vélo

## Extract and transform
From API [https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/](https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/api/?disjunctive.id_compteur&disjunctive.nom_compteur&disjunctive.id&disjunctive.name)


In [6]:
url="https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptage-velo-donnees-compteurs/records"

In [7]:
def updateCyclistData(url, data, offset):
    params = {
    "select": ["sum_counts", "date"],
    "limit": 100,
    "offset": offset,
    "refine": ['nom_compteur:36 quai de Grenelle NE-SO']
    }

    response = json.loads(requests.get(url, params=params).content)

    data["date"] = data["date"] + [dic["date"] for dic in response["results"]]
    data["sum_count"] = data["sum_count"] + [dic["sum_counts"] for dic in response["results"]]

def getTotalCount(url):
    params = {
    "select": ["sum_counts", "date"],
    "limit": 0,
    "offset": 0,
    "refine": ['nom_compteur:36 quai de Grenelle NE-SO']
    }

    response = json.loads(requests.get(url, params=params).content)

    return response["total_count"]

In [8]:
data = {
    "date": [],
    "sum_count": [],
}
offset=0
while offset <= getTotalCount(url):
    updateCyclistData(url, data, offset)
    offset += 100

## Load

In [9]:
df_bike = pd.DataFrame(data)
df_bike["date"] = pd.to_datetime(df_bike["date"])
df_bike = df_bike.set_index("date")
df_bike_1d = df_bike.resample("1d").sum()

In [10]:
df_bike_1d.to_csv(f"../data/cyclist.csv", index=True)

# SQL Lite

## Load and merge

In [11]:
temperature = pd.read_csv("../data/temperature_2015_2024.csv")
cyclist = pd.read_csv("../data/cyclist.csv")

In [12]:
cyclist["date"] = pd.to_datetime(pd.to_datetime(cyclist["date"]).dt.date)
cyclist = cyclist.set_index("date")

temperature["date"] = pd.to_datetime(temperature["date"], format=r'%d-%m-%Y')
temperature = temperature.set_index("date")

In [13]:
df = cyclist.merge(
    temperature,
    how='inner',
    left_index=True,
    right_index=True
)

In [14]:
df = df.rename(columns={
    "sum_count":"nombre de cycliste par jour",
    "temperature max": "temperature_max",
    "temperature min": "temperature_min",
})

In [15]:
df.columns

Index(['nombre de cycliste par jour', 'temperature_max', 'temperature_min'], dtype='object')

## Database

In [18]:
conn = sqlite3.connect('paris_cyclist_temperature.db')
df.to_sql('counters', conn, index=True, if_exists='replace')
conn.close()

# Check the db

In [19]:
conn = sqlite3.connect('paris_cyclist_temperature.db') 
display(pd.read_sql_query("SELECT * FROM counters WHERE temperature_max>10 LIMIT 5", conn))
conn.close()

,date,nombre de cycliste par jour,temperature_max,temperature_min
0,2024-11-01 00:00:00,1389,10.3,7.1
1,2024-11-04 00:00:00,2947,12.7,6.6
2,2024-11-05 00:00:00,3125,11.7,6.4
3,2024-11-09 00:00:00,1822,10.5,7.5
4,2024-11-11 00:00:00,1558,10.7,6.5
